# Initial Join for Model Data: AWS EC2 Notebook
This notebook is meant to be run on an AWS EC2 instance and will produce a combined dataset using the following datasets:
* Base dataset made up of node and edge characteristics and target variable (hex_id, collision year/month/day of week/hour).
* Prior year accident data information for each hex_id and year
* Train-Test-Validation (TTV) split column (hex_id, collision year/month/day of week/hour, accident id, ttv_split)
* Weather data

In [ ]:
import pandas as pd
import numpy as np

import datetime
from datetime import datetime

import boto3

import re

import os
from os.path import isfile, join
from pathlib import Path
from os import listdir
import os
os.getcwd()

#from commons import download_data, find_vcs_root
path =  Path(os.getcwd())
root = path.parent.absolute()
# set name of S3 bucket
s3_bucket = 'traffic-data-bucket'

root


## 1. Import Base data
This includes most of the base data for modeling
* Node and edge information (intersections and streets.
* Includes the accident data as well.

In [ ]:
base_df = pd.read_csv(f"s3://{s3_bucket}/joined_data/base_location_data.csv")

In [ ]:
base_df.sample(5)

## 2. Import Prior Years Accident Information
Various stats regarding collision history for the hexegon and its neighbors

In [ ]:
prior_collision_hist_dict = {}
coll_year_dict = [2014, 2015, 2016, 2017, 2018, 2019, 2020]

for key in coll_year_dict:
    print(key)
    
    raw_path = f"s3://{s3_bucket}/joined_data/{'base_location_' + str(key) + '_collision_data.csv'}"
    prior_collision_hist_dict[key] = pd.read_csv(raw_path)


In [ ]:
for key in coll_year_dict:
    temp_df = prior_collision_hist_dict[key]
    temp_df['collision_year'] = key
    prior_collision_hist_dict[key] = temp_df

In [ ]:
#stack the dictionary of pd frames
prior_collision_hist_df = pd.concat(prior_collision_hist_dict.values(), ignore_index=True)

In [ ]:
prior_collision_hist_df.sample(3)

## 3. Import Train-Test-Validation split data
This includes all collision date and time
* accident_count = 1 is a positive sample
* accident_count = 0 is a negative sample
* ttv_split values are Train, Test, Validate

In [ ]:
ttv_df = pd.read_csv(f"s3://{s3_bucket}/TTV_splits/TTV_data.csv")

In [ ]:
ttv_df.sample()

## 4. Import weather
Weather data for LA county.

In [ ]:
weather_df = pd.read_csv(f"s3://{s3_bucket}/weather/LA_weather_data_updated.csv")

In [ ]:
weather_df.sample(2)

## 4. Impot amenities
Open Streets information for counts
* Restaurants, bars, colleges and schools

In [ ]:
ammenities_df = pd.read_csv(f"s3://{s3_bucket}/nodes_and_edges/la_county_amenities/la_county_ammenities.csv")

In [ ]:
ammenities_df.sample(2)

## 5. Join datasets
* train-test-validate (TTV) split had hex id and date/hour 
* base data - join on hex id
* collision history - join on hex id, collision year


### 5.1 Join train-test-validation data with base data

In [ ]:
joined_df1 = ttv_df[['hex_id','collision_date', 'collision_year','collision_month',
                     'collision_dayofweek','collision_hour',
                     'accident_count','ttv_split']].merge(base_df, on = 'hex_id', how = 'left')

In [ ]:
#should have the same number of rows
ttv_df.shape[0] == joined_df1.shape[0]

### 5.2 Join data prior collision history data

In [ ]:
joined_df2 = joined_df1.merge(prior_collision_hist_df, on = ['hex_id', 'collision_year'], how = 'left')

In [ ]:
#should have the same number of rows
joined_df2.shape[0] == joined_df1.shape[0]

Convert features to date type

In [ ]:
#make sure they are the same format
joined_df2['collision_date'] = pd.to_datetime(joined_df2['collision_date']).dt.date
weather_df['date'] = pd.to_datetime(weather_df['date']).dt.date 

### 5.3 Join data with weather data

In [ ]:
joined_df3 = joined_df2.merge(weather_df, left_on = ['collision_date'], right_on = ['date'], how = 'left')

In [ ]:
#should have the same number of rows
joined_df2.shape[0] == joined_df3.shape[0]

### 5.4 Join data with amenities data

In [ ]:
joined_df4 = joined_df3.merge(ammenities_df, on = ['hex_id'], how = 'left')

In [ ]:
#should have the same number of rows
joined_df4.shape[0] == joined_df3.shape[0]

In [ ]:
joined_df4.sample(3)

In [ ]:
joined_df4.shape

## 6. Save to parquet and upload to S3 bucket

In [ ]:
joined_df4.to_parquet('s3://traffic-data-bucket/model_data/model_data_pre_transformation.parquet', 
                      index=False,engine='pyarrow')

## 6a. Create a local copy of sample data

In [ ]:
#create a local copy
joined_df4[0:100].to_csv(root / 'X.data' / 'model_data' / 'model_data_pre_transformation.csv', index = False )